In [39]:
import requests
from pyvi import ViTokenizer
text = "Chính phủ đang trên đà phát triển#$Bóng đá"
num = "1#$2"
re = requests.post("http://localhost:8000/TextClassification", data={"text":text, 'id':num})
re.json()['size']

4

In [26]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
spark = SparkSession.Builder() \
     .appName("articles") \
     .getOrCreate()
static = spark.read.parquet("hdfs://10.3.71.86:8020/Data/Logging/kinghub/app/2020-07-01")
dataSchema = static.schema
static.show()

+----------+-------------+-------------------+-------+---------+------------------+------------------+--------+-----------+--------------------+------+---------+------------+-----------+-----------------+--------------------+-------+--------------------+----------+--------+-------------+---------------+------+-----+-----+-----+--------------------+-----------------+-----------------+--------------------+-------+-------+-------------+-------+-------+--------+-------+
|serverTime|   clientTime|                 dt|eventId|isVirtual|            postId|            itemId|cardType|contentType|               extra|osType|osVersion|manufacturer|deviceModel|deviceBuildNumber|            deviceId|carrier|               appId|appVersion|language|networkStatus|             ip|pageId|boxId|tagId|catId|             videoId|           userId|      userIdOwner|           requestId|referer|logType|   pageLoadId|groupId|boardId|provider|  boost|
+----------+-------------+-------------------+-------+----

In [27]:
static.printSchema()

root
 |-- serverTime: long (nullable = true)
 |-- clientTime: long (nullable = true)
 |-- dt: string (nullable = true)
 |-- eventId: integer (nullable = true)
 |-- isVirtual: integer (nullable = true)
 |-- postId: string (nullable = true)
 |-- itemId: string (nullable = true)
 |-- cardType: integer (nullable = true)
 |-- contentType: integer (nullable = true)
 |-- extra: string (nullable = true)
 |-- osType: integer (nullable = true)
 |-- osVersion: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- deviceModel: string (nullable = true)
 |-- deviceBuildNumber: string (nullable = true)
 |-- deviceId: string (nullable = true)
 |-- carrier: string (nullable = true)
 |-- appId: string (nullable = true)
 |-- appVersion: string (nullable = true)
 |-- language: string (nullable = true)
 |-- networkStatus: string (nullable = true)
 |-- ip: string (nullable = true)
 |-- pageId: long (nullable = true)
 |-- boxId: string (nullable = true)
 |-- tagId: string (nullable = true

In [28]:
streaming = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 100)\
.parquet("hdfs://10.3.71.86:8020/Data/Logging/kinghub/app/2020-07-01")

In [41]:
activityCounts = streaming.
# activityCounts = streaming

In [40]:
type(streaming)

pyspark.sql.dataframe.DataFrame

In [42]:
spark.conf.set("spark.sql.shuffle.partitions", 5)

In [43]:
activityQuery = activityCounts.writeStream.queryName("label_counts")\
.format("memory").outputMode("complete")\
.start()
# activityQuery.show()

AnalysisException: Complete output mode not supported when there are no streaming aggregations on streaming DataFrames/Datasets;;
FileSource[hdfs://10.3.71.86:8020/Data/Logging/kinghub/app/2020-07-01]

In [36]:
activityQuery.awaitTermination(timeout=1)

False

In [37]:
list(spark.streams.active)

In [38]:
# activityQuery.awaitTermination()
from time import sleep
for x in range(5):
    
    spark.sql("SELECT * FROM label_counts").show()
    sleep(1000)

+--------------------+-----+
|              itemId|count|
+--------------------+-----+
|https://m.kenh14....|   11|
|  727923377931706369|   12|
|https://beta.m.da...|    1|
|   20200630090617764|   44|
|https://m.afamily...|   14|
|https://m.afamily...|    1|
|https://danviet.v...|    2|
|https://tuoitre.v...|  354|
|  727928541317062660|   14|
|https://vietnamne...|    1|
|https://m.afamily...|    1|
|https://thanhnien...|    2|
|https://m.afamily...|    2|
|https://afamily.v...|    8|
|   20200622092043829|   31|
|  727855681089200128|   29|
|https://click.adv...|    2|
|   20190415191244359|  892|
|https://m.afamily...|    2|
|https://m.afamily...|    1|
+--------------------+-----+
only showing top 20 rows



KeyboardInterrupt: 